## Make


In [ ]:
!pip install pandas-highcharts

In [ ]:
import os
import datetime

import IPython
import IPython.display
import requests
import json
import time
import math
import re
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from  matplotlib.ticker import PercentFormatter
import seaborn as sns
 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
 
from sklearn import set_config
# set_config(display='diagram') # Để trực quan hóa pipeline
 
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.utils import resample

from pandas_highcharts.display import display_charts
from pandas_highcharts.display import load_highcharts
from pandas_highcharts.core import serialize
from IPython.core.display import display, HTML
from pandas_highcharts.core import json_encode

In [ ]:
#Set Option
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_theme()

## Data

Food Science Data

We want to introduce Vietnamese food and its nutrition to our foreign friends

Ebook: https://drive.google.com/file/d/0B4RM9E8In1vFX1RYcEFDQ1VwZ1k/view (provided by Ministry of Health of Vietnam - http://chuyentrang.viendinhduong.vn/)

Xls: https://drive.google.com/file/d/0B4RM9E8In1vFS2VVaFBLLUkwZlU/view?resourcekey=0-NrWigVtiVKokEIFJ0HBdcg (Extract by thehinh.com)


## Import Data

In [ ]:
df = pd.read_excel("https://github.com/Al3927/Vietnamese-Food-Nutrition-Analysis/blob/main/data/data.xlsx?raw=true")

In [ ]:
df.head(3)

,TÊN THỨC ĂN,Calories (kcal),Protein (g),Fat (g),Carbonhydrates (g),Chất xơ (g),Cholesterol (mg),Canxi (mg),Photpho (mg),Sắt (mg),Natri (mg),Kali (mg),Beta Caroten (mcg),Vitamin A (mcg),Vitamin B1 (mg),Vitamin C (mg),Loại
0,Gạo nếp cái,346,"8,6","1,5","74,9","0,6",0,32,98,"1,2",3,282,0,0,"0,14","0,0",Ngũ cốc và sản phẩm chế biến từ chúng
1,Gạo tẻ,344,"7,9","1,0","76,2","0,4",0,30,"104,0","1,3",5,241,0,0,"0,10","0,0",Ngũ cốc và sản phẩm chế biến từ chúng
2,Bắp tươi,196,"4,1","2,3","39,6","1,2",0,20,"187,0","1,5",0,0,170,0,"0,21","0,0",Ngũ cốc và sản phẩm chế biến từ chúng


## Data Exploration (simple) & Preprocessing

In [ ]:
df.shape

(162, 17)

### Change column name

In [ ]:
df = df.rename(columns={'TÊN THỨC ĂN': 'Food Name', 'Loại': 'Food Type'})

### Change type col

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Food Name           162 non-null    object
 1   Calories (kcal)     162 non-null    int64 
 2   Protein (g)         162 non-null    object
 3   Fat (g)             162 non-null    object
 4   Carbonhydrates (g)  162 non-null    object
 5   Chất xơ (g)         162 non-null    object
 6   Cholesterol (mg)    162 non-null    int64 
 7   Canxi (mg)          162 non-null    int64 
 8   Photpho (mg)        162 non-null    object
 9   Sắt (mg)            162 non-null    object
 10  Natri (mg)          162 non-null    object
 11  Kali (mg)           162 non-null    int64 
 12  Beta Caroten (mcg)  162 non-null    int64 
 13  Vitamin A (mcg)     162 non-null    object
 14  Vitamin B1 (mg)     162 non-null    object
 15  Vitamin C (mg)      162 non-null    object
 16  Food Type           162 no

In [ ]:
numeric_column = df.select_dtypes(include = 'object').columns.copy()
# numeric_column
numeric_column = np.delete(numeric_column, [0,11])

In [ ]:
def changecol(val):
    if type(val) == str:
        val =  val.replace(",",".")
        val = re.findall(r'\d+',val)[0]
        return val
    else:
        return val

In [ ]:
numeric_column

Index(['Protein (g)', 'Fat (g)', 'Carbonhydrates (g)', 'Chất xơ (g)',
       'Photpho (mg)', 'Sắt (mg)', 'Natri (mg)', 'Vitamin A (mcg)',
       'Vitamin B1 (mg)', 'Vitamin C (mg)'],
      dtype='object')

In [ ]:

for col in numeric_column:
    # df[col] = df[col].agg(changecol)
    df[col] = df.apply(lambda row: changecol(row[col]), axis=1)
    df[col] = df[col].astype(float)

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Food Name           162 non-null    object 
 1   Calories (kcal)     162 non-null    int64  
 2   Protein (g)         162 non-null    float64
 3   Fat (g)             162 non-null    float64
 4   Carbonhydrates (g)  162 non-null    float64
 5   Chất xơ (g)         162 non-null    float64
 6   Cholesterol (mg)    162 non-null    int64  
 7   Canxi (mg)          162 non-null    int64  
 8   Photpho (mg)        162 non-null    float64
 9   Sắt (mg)            162 non-null    float64
 10  Natri (mg)          162 non-null    float64
 11  Kali (mg)           162 non-null    int64  
 12  Beta Caroten (mcg)  162 non-null    int64  
 13  Vitamin A (mcg)     162 non-null    float64
 14  Vitamin B1 (mg)     162 non-null    float64
 15  Vitamin C (mg)      162 non-null    float64
 16  Food Typ

### Distribution of numeric feature

In [ ]:
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25)
def median(df):
    return df.quantile(0.5)
def upper_quartile(df):
    return df.quantile(0.75)

numeric_column = df.select_dtypes(exclude = 'object').columns
num_df = df[numeric_column].copy()
num_col_info_df = num_df.agg([missing_ratio, min, lower_quartile, median, upper_quartile, max]).round(1)
num_col_info_df


,Calories (kcal),Protein (g),Fat (g),Carbonhydrates (g),Chất xơ (g),Cholesterol (mg),Canxi (mg),Photpho (mg),Sắt (mg),Natri (mg),Kali (mg),Beta Caroten (mcg),Vitamin A (mcg),Vitamin B1 (mg),Vitamin C (mg)
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lower_quartile,89.0,4.0,0.0,0.0,0.0,0.0,8.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median,138.0,13.0,3.0,1.0,0.0,0.0,21.5,134.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
upper_quartile,326.8,18.0,11.8,16.5,0.0,0.0,51.5,210.0,2.0,0.0,0.0,0.0,9.5,0.0,0.0
max,680.0,75.0,59.0,99.0,8.0,1790.0,5040.0,995.0,52.0,4926.0,1132.0,1470.0,6960.0,0.0,41.0


### Distribution of object feature

In [ ]:
#Tham khảo [1]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
object_df = df.select_dtypes(include='object')
cat_col_info_df = object_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,Food Name,Food Type
missing_ratio,0,0
num_values,162,11
value_ratios,"{'Đậu phộng chiên': 0.6, 'Bánh tráng mỏng': 0.6, 'Cá nục': 0.6, 'Cá rô đồng': 0.6, 'Cá thu hộp': 0.6, 'Nem chua': 0.6, 'Cá ngừ': 0.6, 'Tôm đồng': 0.6, 'Bánh sôcôla': 0.6, 'Thịt heo mỡ': 0.6, 'Lòng trắng trứngvịt': 0.6, 'Óc heo': 0.6, 'Hải sâm': 0.6, 'Bánh phở': 0.6, 'Sườn heo bỏ xương': 0.6, 'Tim heo': 0.6, 'Thịt heo ba chỉ': 0.6, 'Đậu phụ': 0.6, 'Lưỡi bò': 0.6, 'Đậu Hà lan (hạt)': 0.6, 'Kẹo đậu phộng': 0.6, 'Đuôi heo': 0.6, 'Tim gà': 0.6, 'Lưỡi heo': 0.6, 'Trứng gà': 0.6, 'Mực khô': 0.6, 'Cá hồi': 0.6, 'Thịt heo hộp': 0.6, 'Sò': 0.6, 'Bánh in chay': 0.6, 'Khoai tây chiên': 0.6, 'Pa-tê': 0.6, 'Cá bống': 0.6, 'Nhãn hộp': 0.6, 'Kẹo dừa mềm': 0.6, 'Rượu trắng': 0.6, 'Bao tử heo': 0.6, 'Thịt dê nạc': 0.6, 'Cá phèn': 0.6, 'Tôm chua': 0.6, 'Bột sắn dây': 0.6, 'Nước mắm': 0.6, 'Huyết bò': 0.6, 'Tai heo': 0.6, 'Cá lóc': 0.6, 'Cá chép': 0.6, 'Bia': 0.6, 'Nước thơm': 0.6, 'Lòng đỏ trứng vịt': 0.6, 'Óc bò': 0.6, 'Thịt thỏ': 0.6, 'Gừng tươi': 0.6, 'Mứt thơm': 0.6, 'Tôm khô': 0.6, 'Đuôi bò': 0.6, 'Mắm tôm đặc': 0.6, 'Củ từ': 0.6, 'Sữa bò tươi': 0.6, 'Tôm biển': 0.6, 'Lạp xưởng': 0.6, 'Bánh mì khô': 0.6, 'Mực tươi': 0.6, 'Thịt gà ta': 0.6, 'Cari bột': 0.6, 'Khoai lang': 0.6, 'Cá mỡ': 0.6, 'Huyết heo luộc': 0.6, 'Cá nạc': 0.6, 'Cá rô phi': 0.6, 'Giòheo bỏ xương': 0.6, 'Khoai môn': 0.6, 'Cá mè': 0.6, 'Cùi dừa già': 0.6, 'Tim bò': 0.6, 'Xúc xích': 0.6, 'Rượu nếp': 0.6, 'Cua đồng': 0.6, 'Gan gà': 0.6, 'Cá trê': 0.6, 'Bánh men': 0.6, 'Đầu heo': 0.6, 'Cật bò': 0.6, 'Thịt vai chó': 0.6, 'Xì dầu': 0.6, 'Cật heo': 0.6, 'Chà bông': 0.6, 'Thịt bò hộp': 0.6, 'Phèo heo': 0.6, 'Hến': 0.6, 'Chả lụa': 0.6, 'Muối': 0.6, 'Cá khô': 0.6, 'Kẹo cà phê': 0.6, 'Nhộng': 0.6, 'Cá trích hộp': 0.6, 'Lòngheo (ruột già)': 0.6, 'Đường cát trắng': 0.6, 'Gan vịt': 0.6, 'Đậu phộng': 0.6, 'Kẹo sữa': 0.6, ...}","{'THỊT VÀ SẢN PHẨM CHẾ BIẾN': 32.7, 'THỦY SẢN VÀ SẢN PHẨM CHẾ BIẾN': 21.0, 'ĐỒ NGỌT (ĐƯỜNG, BÁNH, MỨT, KẸO)': 8.0, 'ĐỒ HỘP': 7.4, 'GIA VỊ, NƯỚC CHẤM': 6.2, 'Hạt, quả giàu protein, chất béo và chế phẩm': 5.6, 'Ngũ cốc và sản phẩm chế biến từ chúng': 5.6, 'Khoai củ và các sản phẩm chế biến từ chúng': 5.6, 'TRỨNG VÀ SẢN PHẨM CHẾ BIẾN': 4.3, 'NƯỚC GIẢI KHÁT': 2.5, 'SỮA VÀ SẢN PHẨM CHẾ BIẾN': 1.2}"


## Analysis

### Compare some common food

In [ ]:
common_food = ['Bánh mì', 'Gạo tẻ', 'Bún', 'Đậu phụ', 'Thịt bò', 'Thịt gà ta', 'Thịt heo ba chỉ', 'Chả lụa', 'Xúc xích', 'Cá ngừ', 'Cá rô phi', 'Cá thu', 'Tôm biển', 'Trứng gà', 'Sữa bò tươi', 'Đường cát trắng', 'Muối', 'Nước mắm', 'Xì dầu', 'Bia']

In [ ]:
cfood_df = pd.DataFrame(columns = df.columns.copy())
for cfood in common_food:
    cfood_df = cfood_df.append(df[df['Food Name'] == cfood].copy())

In [ ]:
cfood_df.shape

(20, 17)

In [100]:
cfood_col = cfood_df.select_dtypes(include= 'object').columns.copy()
cfood_col = np.delete(cfood_col,[0,6])
cfood_col
for col in cfood_col:
    cfood_df[col] = cfood_df[col].astype(float)


In [ ]:
def draw_bar (data_frame, renderTo, title_name, subtitle, typeData = 'default', header_tooltip = 'header'):
  data_frame_visualization = serialize(data_frame, render_to=renderTo, output_type="dict")
  data_frame_visualization["title"] = {"text" : {title_name}}
  data_frame_visualization["subtitle"] = {"text": f"{subtitle}"}
  data_frame_visualization["chart"]["type"] = "column"
  data_frame_visualization["xAxis"]["type"] = "category"

  if (typeData == 'default'):
    data_frame_visualization["plotOptions"] = {"series": {"borderWidth": 0,"dataLabels": {"enabled": "true"}}}
    data_frame_visualization["tooltip"] = {
        "headerFormat": '<span style="font-size:11px"><b>{header}</b></span><br>'.format(header=header_tooltip), 
        "pointFormat": '<span style="color:{point.color}">{point.name}</span>: <b>{point.y}</b><br/>'}
  else:
    if (typeData == 'rate'):
      data_frame_visualization["plotOptions"] = {"series": 
                                                 {"borderWidth": 0,"dataLabels": 
                                                  {"enabled": "true", "format": '{point.y:.2f}%'}}}
      data_frame_visualization["tooltip"] = {
          "headerFormat": '<span style="font-size:11px"><b>{header}</b></span><br>'.format(header=header_tooltip),
          "pointFormat": '<span style="color:{point.color}">{point.name}</span>: <b>{point.y:.2f}%</b> of total<br/>'}
    else:
      data_frame_visualization["plotOptions"] = {"series": 
                                                 {"borderWidth": 0,"dataLabels": 
                                                  {"enabled": "true", "format": '{point.y:.2f}'}}}
      data_frame_visualization["tooltip"] = {
        "headerFormat": '<span style="font-size:11px"><b>{header}</b></span><br>'.format(header=header_tooltip), 
        "pointFormat": '<span style="color:{point.color}">{point.name}</span>: <b>{point.y:.2f}</b><br/>'}

  json_data_frame= "new Highcharts.Chart(%s);" % json_encode(data_frame_visualization)

  display(HTML(
    """
    <meta charset="utf-8"> 
    <div id="{chart_id}"</div>
    <script src="//code.highcharts.com/stock/highstock.js"></script><script src="//code.highcharts.com/highcharts-more.js"></script>
    <script src="//code.highcharts.com/modules/exporting.js"></script><script type="text/javascript">{data}</script>        
    """.format(chart_id=renderTo, data=json_data_frame)))

In [102]:
for column in cfood_df.select_dtypes(exclude='object').columns:
    cfood_df_ = cfood_df[['Food Name',f'{column}']].copy()
    cfood_df_ = cfood_df_.sort_values(by = [f'{column}'], ascending=True)
    cfood_df_ = cfood_df_.set_index('Food Name')
    draw_bar(cfood_df_,
            header_tooltip=f'{column}',
            renderTo=f'cfood_{column}', 
            title_name=f'Distribution of {column}',
            subtitle='of common food')